In [1]:
#Python dependencies
import os
import sys

# Standard dependencies
import cv2
import time
import scipy as sp
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt

# Machine Learning
import tensorflow as tf
import keras
from keras.models import Model
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K
from keras.activations import elu
from keras.optimizers import Adam
from keras.models import Sequential
from keras.engine import Layer, InputSpec
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, LeakyReLU, GlobalAveragePooling2D
from keras.activations import relu, softmax, sigmoid, linear, elu
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import cohen_kappa_score
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
test_data = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_data.head(5)

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [3]:
# Specify image size
IMG_WIDTH = 528
IMG_HEIGHT = 528
CHANNELS = 3
BATCH = 4

In [4]:
def crop_image_from_gray(img, tol=7):
    """
    Applies masks to the orignal image and 
    returns the a preprocessed image with 
    3 channels
    """
    # If for some reason we only have two channels
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    # If we have a normal RGB images
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def preprocess_image(img, sigmaX=10):
    """
    The whole preprocessing pipeline:
    Resize image to desired size
    """
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = crop_image_from_gray(img)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return img

In [5]:
test_data['id_code'] = test_data['id_code'] + '.png'
test_data.head(5)

,id_code,diagnosis
0,0005cfc8afb6.png,0
1,003f0afdcd15.png,0
2,006efc72b638.png,0
3,00836aaacf06.png,0
4,009245722fa4.png,0


In [6]:
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    
    :param model: A Keras model object
    :param generator: A Keras ImageDataGenerator object
    
    :return: A tuple with two Numpy Arrays. One containing the predictions
    and one containing the labels
    """
    preds = []
    for _ in range(int(np.ceil(generator.samples / BATCH))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel()

In [7]:
BATCH_SIZE = 12
from keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(vertical_flip=True, horizontal_flip=True,rotation_range=360, preprocessing_function=preprocess_image, rescale=1 / 255.)
datagen = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1/255.)

test_generator = datagen.flow_from_dataframe(test_data, 
                                                x_col='id_code', 
                                                y_col=None,
                                                directory = '../input/aptos2019-blindness-detection/test_images/',
                                                target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                batch_size=BATCH_SIZE,
                                                class_mode=None,
                                                shuffle = False)

Found 1928 validated image filenames.


In [8]:
sys.path.append(os.path.abspath('../input/effnet/efficientnet-master/'))
from efficientnet.keras import EfficientNetB6

In [9]:
# Load json and create model
json_file = open('../input/regressb6/model_b6.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights("../input/regressb6/weights_kappa_b6.h5")
print("Loaded model from disk")

loaded_model.compile(loss='mse',optimizer=Adam(lr=0.0001))


Loaded model from disk


In [10]:
class RegressionRounder(object):
    """
    Rounder for regression
    """
    def __init__(self):
        self.coef0 = 0.5
        self.coef1 = 1.5
        self.coef2 = 2.5
        self.coef3 = 3.5

    def predict(self, X):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < self.coef0:
                X_p[i] = 0
            elif pred >= self.coef0 and pred < self.coef1:
                X_p[i] = 1
            elif pred >= self.coef1 and pred < self.coef2:
                X_p[i] = 2
            elif pred >= self.coef2 and pred < self.coef3:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

In [11]:
# Make final predictions, round predictions and save to csv
RR = RegressionRounder()

predict = loaded_model.predict_generator(test_generator)
predict = RR.predict(predict)
test_data['diagnosis'] = predict.astype('int')
# # Remove .png from ids
test_data['id_code'] = test_data['id_code'].str.replace(r'.png$', '')
test_data.to_csv('submission.csv', index=False)

In [12]:
test_data.head(30)

,id_code,diagnosis
0,0005cfc8afb6,1
1,003f0afdcd15,3
2,006efc72b638,3
3,00836aaacf06,1
4,009245722fa4,3
5,009c019a7309,2
6,010d915e229a,2
7,0111b949947e,0
8,01499815e469,3
9,0167076e7089,0
